In [53]:
from plotly import express as px
import pandas as pd
from sklearn.metrics import roc_curve
import numpy as np

In [15]:
ab_df = pd.read_csv('ab_model_predictions.csv')
ab_df.rename({'final_predictions_adaboost': 'Image Features (AdaBoost)'}, axis=1, inplace=True)
results_df.head()

,path,y_x,final_predictions_adaboost,y_y,Model Class Prediction,Model Class Score
0,cda-project/images/Test/n/1_1_36_2_0.jpg,False,0.496786,0,0,2.036121e-07
1,cda-project/images/Test/n/1_1_36_3_0.jpg,False,0.496908,0,0,8.946770e-06
2,cda-project/images/Test/n/1_1_36_4_0.jpg,False,0.497927,0,1,9.009228e-01
3,cda-project/images/Test/n/1_1_36_5_0.jpg,False,0.499158,0,0,1.493833e-20
4,cda-project/images/Test/n/1_1_37_2_0.jpg,False,0.486900,0,0,4.546106e-02


In [16]:
cnn_df = pd.read_csv('cnn_model_predictions.csv')
cnn_df.rename({'Model Class Score': 'CNN'}, axis=1, inplace=True)
cnn_df.head()

,path,y,Model Class Prediction,CNN
0,cda-project/images/Test/p/1_1_36_1_0.jpg,1,1,0.992121
1,cda-project/images/Test/p/1_1_36_1_1.jpg,1,1,0.995521
2,cda-project/images/Test/p/1_1_36_1_2.jpg,1,1,0.992097
3,cda-project/images/Test/p/1_1_36_1_3.jpg,1,1,0.993814
4,cda-project/images/Test/p/1_1_37_1_0.jpg,1,1,0.967347


In [17]:
results_df = ab_df.merge(cnn_df, how='outer', on='path')

In [18]:
results_df.isna().sum()

path                         0
y_x                          0
Image Features (AdaBoost)    0
y_y                          0
Model Class Prediction       0
CNN                          0
dtype: int64

In [19]:
results_df.head()

,path,y_x,Image Features (AdaBoost),y_y,Model Class Prediction,CNN
0,cda-project/images/Test/n/1_1_36_2_0.jpg,False,0.496786,0,0,2.036121e-07
1,cda-project/images/Test/n/1_1_36_3_0.jpg,False,0.496908,0,0,8.946770e-06
2,cda-project/images/Test/n/1_1_36_4_0.jpg,False,0.497927,0,1,9.009228e-01
3,cda-project/images/Test/n/1_1_36_5_0.jpg,False,0.499158,0,0,1.493833e-20
4,cda-project/images/Test/n/1_1_37_2_0.jpg,False,0.486900,0,0,4.546106e-02


In [27]:
fpr, tpr, threshold = roc_curve(ab_df.y, ab_df['Image Features (AdaBoost)'])

ab_plot_df = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': threshold})
ab_plot_df['Model'] = 'AdaBoost on Distortion Features'

In [28]:
fpr, tpr, threshold = roc_curve(cnn_df.y, cnn_df['CNN'])

cnn_plot_df = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': threshold})
cnn_plot_df['Model'] = 'CNN on Images'

In [29]:
plot_df = pd.concat([ab_plot_df, cnn_plot_df])

In [35]:
roc_fig = px.line(plot_df, x='FPR', y='TPR', color='Model', title='ROC Curve', width=900)
roc_fig

In [37]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [43]:
print(classification_report(ab_df.y, ab_df['Image Features (AdaBoost)']>0.5))
print(confusion_matrix(ab_df.y, ab_df['Image Features (AdaBoost)']>0.5))
print(accuracy_score(ab_df.y, ab_df['Image Features (AdaBoost)']>0.5))
print(roc_auc_score(ab_df.y, ab_df['Image Features (AdaBoost)']))

precision    recall  f1-score   support

       False       0.71      0.78      0.74      1277
        True       0.75      0.68      0.71      1256

    accuracy                           0.73      2533
   macro avg       0.73      0.73      0.73      2533
weighted avg       0.73      0.73      0.73      2533

[[991 286]
 [408 848]]
0.726016581129096
0.7955904064562146


In [45]:
print(classification_report(cnn_df.y, cnn_df['CNN']>0.5))
print(confusion_matrix(cnn_df.y, cnn_df['CNN']>0.5))
print(accuracy_score(cnn_df.y, cnn_df['CNN']>0.5))
print(roc_auc_score(cnn_df.y, cnn_df['CNN']))

precision    recall  f1-score   support

           0       0.95      0.72      0.82      1277
           1       0.77      0.96      0.86      1256

    accuracy                           0.84      2533
   macro avg       0.86      0.84      0.84      2533
weighted avg       0.86      0.84      0.84      2533

[[ 919  358]
 [  49 1207]]
0.8393209632846427
0.9261733187356911


In [39]:
help(classification_report)

Help on function classification_report in module sklearn.metrics._classification:

classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')
    Build a text report showing the main classification metrics
    
    Read more in the :ref:`User Guide <classification_report>`.
    
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.
    
    y_pred : 1d array-like, or label indicator array / sparse matrix
        Estimated targets as returned by a classifier.
    
    labels : array, shape = [n_labels]
        Optional list of label indices to include in the report.
    
    target_names : list of strings
        Optional display names matching the labels (same order).
    
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    
    digits : int
        Number of digits for for

## Parameter Tuning

In [1]:
rf_results = '''
0.663229 (0.071176) with: {'max_depth': 10, 'n_estimators': 50}
0.665317 (0.074164) with: {'max_depth': 10, 'n_estimators': 100}
0.665944 (0.071151) with: {'max_depth': 10, 'n_estimators': 150}
0.666570 (0.072041) with: {'max_depth': 10, 'n_estimators': 200}
0.662811 (0.070084) with: {'max_depth': 10, 'n_estimators': 250}
0.663019 (0.074380) with: {'max_depth': 10, 'n_estimators': 300}
0.663228 (0.071527) with: {'max_depth': 10, 'n_estimators': 350}
0.662183 (0.074257) with: {'max_depth': 10, 'n_estimators': 400}
0.660302 (0.076942) with: {'max_depth': 10, 'n_estimators': 450}
0.662382 (0.138479) with: {'max_depth': 20, 'n_estimators': 50}
0.662175 (0.134311) with: {'max_depth': 20, 'n_estimators': 100}
0.659877 (0.136313) with: {'max_depth': 20, 'n_estimators': 150}
0.663223 (0.128946) with: {'max_depth': 20, 'n_estimators': 200}
0.662595 (0.130702) with: {'max_depth': 20, 'n_estimators': 250}
0.662176 (0.132829) with: {'max_depth': 20, 'n_estimators': 300}
0.662594 (0.132322) with: {'max_depth': 20, 'n_estimators': 350}
0.665100 (0.131697) with: {'max_depth': 20, 'n_estimators': 400}
0.664892 (0.130758) with: {'max_depth': 20, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 30, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 30, 'n_estimators': 100}
0.660504 (0.138254) with: {'max_depth': 30, 'n_estimators': 150}
0.664265 (0.134622) with: {'max_depth': 30, 'n_estimators': 200}
0.662385 (0.132953) with: {'max_depth': 30, 'n_estimators': 250}
0.663430 (0.131983) with: {'max_depth': 30, 'n_estimators': 300}
0.663639 (0.134644) with: {'max_depth': 30, 'n_estimators': 350}
0.662804 (0.132232) with: {'max_depth': 30, 'n_estimators': 400}
0.663639 (0.131995) with: {'max_depth': 30, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 40, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 40, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 40, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 40, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 40, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 40, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 40, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 40, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 40, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 50, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 50, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 50, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 50, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 50, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 50, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 50, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 50, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 50, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 60, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 60, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 60, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 60, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 60, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 60, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 60, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 60, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 60, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 70, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 70, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 70, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 70, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 70, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 70, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 70, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 70, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 70, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 80, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 80, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 80, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 80, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 80, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 80, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 80, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 80, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 80, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 90, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 90, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 90, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 90, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 90, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 90, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 90, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 90, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 90, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 100, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 100, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 100, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 100, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 100, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 100, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 100, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 100, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 100, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 110, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 110, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 110, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 110, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 110, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 110, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 110, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 110, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 110, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 120, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 120, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 120, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 120, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 120, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 120, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 120, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 120, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 120, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 130, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 130, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 130, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 130, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 130, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 130, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 130, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 130, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 130, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 140, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 140, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 140, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 140, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 140, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 140, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 140, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 140, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 140, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 150, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 150, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 150, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 150, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 150, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 150, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 150, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 150, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 150, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 160, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 160, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 160, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 160, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 160, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 160, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 160, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 160, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 160, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 170, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 170, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 170, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 170, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 170, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 170, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 170, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 170, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 170, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 180, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 180, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 180, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 180, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 180, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 180, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 180, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 180, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 180, 'n_estimators': 450}
0.652983 (0.140662) with: {'max_depth': 190, 'n_estimators': 50}
0.658206 (0.140275) with: {'max_depth': 190, 'n_estimators': 100}
0.661757 (0.139212) with: {'max_depth': 190, 'n_estimators': 150}
0.664265 (0.134228) with: {'max_depth': 190, 'n_estimators': 200}
0.663430 (0.133680) with: {'max_depth': 190, 'n_estimators': 250}
0.663221 (0.132158) with: {'max_depth': 190, 'n_estimators': 300}
0.662803 (0.134526) with: {'max_depth': 190, 'n_estimators': 350}
0.663848 (0.132779) with: {'max_depth': 190, 'n_estimators': 400}
0.663430 (0.132168) with: {'max_depth': 190, 'n_estimators': 450}'''

In [46]:
rf_list = rf_results.split('\n')

In [54]:
rf_list[1]
rf_split = np.array([x.split(' ') for x in rf_list[1:]])

In [78]:
rf_df = pd.DataFrame(rf_split[:,[0, 1, 4, 6]], columns=['Mean Accuracy', 'Accuracy Std. Dev.', 'Max Depth', 'Number of Trees'] )
rf_df['Accuracy Std. Dev.'] = rf_df['Accuracy Std. Dev.'].apply(lambda x: float(x[1:-1]))
rf_df['Max Depth'] = rf_df['Max Depth'].apply(lambda x: x[:-1])
rf_df['Number of Trees'] = rf_df['Number of Trees'].apply(lambda x: int(x[:-1]))
rf_df['Mean Accuracy'] = rf_df['Mean Accuracy'].astype(float)

In [79]:
main_fig = px.line(rf_df, x='Number of Trees', y='Mean Accuracy', color='Max Depth')
scatter_fig = px.scatter(rf_df, x='Number of Trees', y='Mean Accuracy', color='Max Depth')
scatter_fig.update_layout(showlegend=False)
for trace in scatter_fig.data:
    main_fig.add_trace(trace)
main_fig

In [70]:
rf_df.head()

,Mean Accuracy,Accuracy Std. Dev.,Max Depth,Number of Trees
0,0.663229,0.071176,10,50}
1,0.665317,0.074164,10,100}
2,0.665944,0.071151,10,150}
3,0.666570,0.072041,10,200}
4,0.662811,0.070084,10,250}


In [2]:
ab_results = '''
0.663868 (0.058265) with: {'n_estimators': 50}
0.670136 (0.048434) with: {'n_estimators': 100}
0.670552 (0.051520) with: {'n_estimators': 150}
0.668880 (0.049946) with: {'n_estimators': 200}
0.670968 (0.052855) with: {'n_estimators': 250}
0.667625 (0.051903) with: {'n_estimators': 300}
0.665536 (0.051994) with: {'n_estimators': 350}
0.660936 (0.059080) with: {'n_estimators': 400}
0.668459 (0.055454) with: {'n_estimators': 450}
0.666998 (0.054713) with: {'n_estimators': 500}
0.660312 (0.060602) with: {'n_estimators': 550}
0.660311 (0.061656) with: {'n_estimators': 600}
0.660937 (0.060788) with: {'n_estimators': 650}
0.661356 (0.060135) with: {'n_estimators': 700}
0.668669 (0.056777) with: {'n_estimators': 750}
0.669924 (0.056573) with: {'n_estimators': 800}
0.670342 (0.054828) with: {'n_estimators': 850}
0.671178 (0.054759) with: {'n_estimators': 900}
0.666165 (0.054720) with: {'n_estimators': 950}'''

In [80]:
ab_list = ab_results.split('\n')

In [82]:
ab_split = np.array([x.split(' ') for x in ab_list[1:]])

array([['0.663868', '(0.058265)', '50}'],
       ['0.670136', '(0.048434)', '100}'],
       ['0.670552', '(0.051520)', '150}'],
       ['0.668880', '(0.049946)', '200}'],
       ['0.670968', '(0.052855)', '250}'],
       ['0.667625', '(0.051903)', '300}'],
       ['0.665536', '(0.051994)', '350}'],
       ['0.660936', '(0.059080)', '400}'],
       ['0.668459', '(0.055454)', '450}'],
       ['0.666998', '(0.054713)', '500}'],
       ['0.660312', '(0.060602)', '550}'],
       ['0.660311', '(0.061656)', '600}'],
       ['0.660937', '(0.060788)', '650}'],
       ['0.661356', '(0.060135)', '700}'],
       ['0.668669', '(0.056777)', '750}'],
       ['0.669924', '(0.056573)', '800}'],
       ['0.670342', '(0.054828)', '850}'],
       ['0.671178', '(0.054759)', '900}'],
       ['0.666165', '(0.054720)', '950}']], dtype='<U16')

In [86]:
ab_df = pd.DataFrame(ab_split[:,[0,1,-1]], columns=['Mean Accuracy', 'Accuracy Std. Dev.', 'Number of Trees'] )

In [87]:
ab_df['Accuracy Std. Dev.'] = ab_df['Accuracy Std. Dev.'].apply(lambda x: float(x[1:-1]))
ab_df['Number of Trees'] = ab_df['Number of Trees'].apply(lambda x: int(x[:-1]))
ab_df['Mean Accuracy'] = ab_df['Mean Accuracy'].astype(float)

In [88]:
ab_df.head()

,Mean Accuracy,Accuracy Std. Dev.,Number of Trees
0,0.663868,0.058265,50
1,0.670136,0.048434,100
2,0.670552,0.051520,150
3,0.668880,0.049946,200
4,0.670968,0.052855,250


In [90]:
main_fig = px.line(ab_df, x='Number of Trees', y='Mean Accuracy')
scatter_fig = px.scatter(ab_df, x='Number of Trees', y='Mean Accuracy')
scatter_fig.update_layout(showlegend=False)
for trace in scatter_fig.data:
    main_fig.add_trace(trace)
main_fig